In [1]:
!pip install konlpy

     ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
     --------------------------------------- 0.0/19.4 MB 435.7 kB/s eta 0:00:45
     --------------------------------------- 0.1/19.4 MB 751.6 kB/s eta 0:00:26
     ----- ---------------------------------- 2.5/19.4 MB 15.8 MB/s eta 0:00:02
     ------------- -------------------------- 6.7/19.4 MB 30.8 MB/s eta 0:00:01
     -------------------- ------------------ 10.1/19.4 MB 38.0 MB/s eta 0:00:01
     --------------------------- ----------- 13.6/19.4 MB 73.1 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 M

In [2]:
!pip install Twitter

  Using cached twitter-1.19.6-py2.py3-none-any.whl (50 kB)


In [3]:
!pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [4]:
!pip install gensim

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/ab/b0/d58dc405fd60ab546ca714321235dc2d455b2dc06bfb4fc1092940c749fc/gensim-4.3.2-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.7.0 from https://files.pythonhosted.org/packages/70/03/485f73046134400ea25d3cb178c5e6728f9b165f79d09638ecb44ee0e9b1/scipy-1.11.2-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ------------------- ------------------ 30.7/59.1 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 59.1/59.1 kB 775.6 kB/s eta 0:00:00
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/24.0 MB ? eta -:--

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
theano-pymc 1.1.2 requires filelock, which is not installed.
arviz 0.12.1 requires matplotlib>=3.0, which is not installed.
arviz 0.12.1 requires xarray>=0.16.1, which is not installed.
xarray-einstats 0.5.1 requires xarray>=2022.09.0, which is not installed.


In [5]:
import os
import pymysql
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from gensim.models.word2vec import Word2Vec

In [6]:
# 본인 컴퓨터의 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [7]:
os.environ["JAVA_HOME"]

'C:\\Program Files\\Java\\jdk-11'

In [8]:
from konlpy.tag import Twitter

In [9]:
# 한국어 문자열을 단어로 나누고 품사를 리턴하는 Twitter() 객체 생성
twitter = Twitter()

c:\Users\tjk\AppData\Local\anaconda3\envs\movie_env01\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [10]:
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워")

[('안녕', 'Noun'),
 ('하십니까', 'Verb'),
 ('좋은', 'Adjective'),
 ('아침', 'Noun'),
 ('입니다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반가워', 'Adjective')]

In [11]:
# stem=True: 동사인 하십니까 반가워는 기본 동사형 '안녕하다'와 '반갑다'로 변환
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워 집으로 뛰어 갔다 정말 참으로 좋다", stem=True)

[('안녕', 'Noun'),
 ('하다', 'Verb'),
 ('좋다', 'Adjective'),
 ('아침', 'Noun'),
 ('이다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반갑다', 'Adjective'),
 ('집', 'Noun'),
 ('으로', 'Josa'),
 ('뛰다', 'Verb'),
 ('가다', 'Verb'),
 ('정말', 'Noun'),
 ('참으로', 'Noun'),
 ('좋다', 'Adjective')]

In [13]:
# 텍스트 전처리 함수
# text는 전처리할 영화 줄거리(synopsys)입니다.
def preprocessingText(text):
    # 전처리 결과를 저장할 리스트
    stems = []
    
    # text를 단어로 나누고 품사를 붙여서 리턴
    tagged_review = twitter.pos(text, stem=True)
    
    # tagged_review에서 하나를 꺼내서 word, pos에 대입
    # 단어는 word에, 품사는 pos에 대입
    for word, pos in tagged_review:
        if pos == "Noun" or pos == "Adjective":
            stems.append(word)
            
    return " ".join(stems)

In [14]:
# 함수 테스트
preprocessingText("아버지가 방에 들어가신다 그래서 슬펐다 배가 아파요")

'아버지 방 슬프다 배 아프다'

In [15]:
# DB 연결
db = pymysql.connect(
    host="database-1.crcwt5epkeh5.ap-northeast-2.rds.amazonaws.com",
    port=3306,
    user='admin',
    passwd='password',
    db='movie_db',
    charset='utf8'
)

In [16]:
# DB 연결 테스트
sql = "SELECT * FROM movie_tbl;"
movie_df = pd.read_sql(sql, db)

C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\1045968588.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_df = pd.read_sql(sql, db)


In [17]:
movie_df

,num,title,poster,degree,genre,open_date,country,movie_time,synopsys,synopsys_vector
0,1,범죄도시3,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"범죄, 액션",2023.05.31.,대한민국,105분,"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 ...",None
1,2,스즈메의 문단속,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.03.08.,일본,122분,“이 근처에 폐허 없니? 문을 찾고 있어” 규슈의 한적한 마을에 살고 있는 소녀 ‘...,None
2,3,엘리멘탈,https://search.pstatic.net/common?type=o&size=...,전체 관람가,애니메이션,2023.06.14.,미국,109분,"불, 물, 공기, 흙 4개의 원소들이 살고 있는 ‘엘리멘트 시티’ 재치 있고 불처럼...",None
3,4,더 퍼스트 슬램덩크,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.01.04.,일본,124분,"전국 제패를 꿈꾸는 북산고 농구부 5인방의 꿈과 열정, 멈추지 않는 도전을 그린 영화",None
4,5,가디언즈 오브 갤럭시: Volume 3,https://search.pstatic.net/common?type=o&size=...,12세 관람가,"액션, 모험, SF",2023.05.03.,미국,150분,‘가모라’를 잃고 슬픔에 빠져 있던 ‘피터 퀼’이 위기에 처한 은하계와 동료를 지키...,None
...,...,...,...,...,...,...,...,...,...,...
622,623,잇 컴스 앳 나잇,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"미스터리, 스릴러",2017.10.19.,미국,92분,의문의 존재를 피해 외딴 숲에 집을 짓고 자신만의 규칙을 만들어 아내와 아들을 돌보...,None
623,624,퍼스널 쇼퍼,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"드라마, 스릴러",2017.02.09.,프랑스,105분,프랑스 파리에서 퍼스널 쇼퍼로 일하는 미국 여자 ‘모린’(크리스틴 스튜어트) 영혼들...,None
624,625,케빈에 대하여,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"드라마, 스릴러, 서스펜스",2012.07.26.,"영국, 미국",112분,자유로운 삶을 즐기던 여행가 에바에게 아들 케빈이 생기면서 그녀의 삶은 180도 달...,None
625,626,악당들,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"범죄, 액션",2023.05.31.,"이탈리아, 브라질",89분,익명의 고객이 의뢰한 일을 완수만 한다면 특별사면을 보장하겠다는 무법 변호사의 제안...,None


In [18]:
movie_df['synopsys'][0]

"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 뒤, ‘마석도’(마동석)는 새로운 팀원들과 함께 살인사건을 조사한다. 사건 조사 중, ‘마석도’는 신종 마약 사건이 연루되었음을 알게 되고 수사를 확대한다. 한편, 마약 사건의 배후인 '주성철'(이준혁)은 계속해서 판을 키워가고 약을 유통하던 일본 조직과 '리키'(아오키 무네타카)까지 한국에 들어오며 사건의 규모는 점점 더 커져가는데... 나쁜 놈들 잡는 데 이유 없고 제한 없다. 커진 판도 시원하게 싹 쓸어버린다!"

In [19]:
preprocessingText(movie_df['synopsys'][0])

'대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마 석도 마동석 새롭다 팀원 살인 사건 조사 사건 조사 중 마 석도 신종 마약 사건 연루 알 수사 확대 한편 마약 사건 배후 주성철 이준혁 은 계속 판 약 유통 일본 조직 리키 아오키 무 타카 한국 사건 규모 점점 더 나쁘다 놈 데 이유 없다 제한 없다 판도 시원하다 싹'

In [20]:
# "synopsys_clear" 컬럼을 추가하고 초기값으로 np.NaN(결측치) 대입
movie_df["synopsys_clear"] = np.NaN

In [21]:
movie_df

,num,title,poster,degree,genre,open_date,country,movie_time,synopsys,synopsys_vector,synopsys_clear
0,1,범죄도시3,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"범죄, 액션",2023.05.31.,대한민국,105분,"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 ...",None,NaN
1,2,스즈메의 문단속,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.03.08.,일본,122분,“이 근처에 폐허 없니? 문을 찾고 있어” 규슈의 한적한 마을에 살고 있는 소녀 ‘...,None,NaN
2,3,엘리멘탈,https://search.pstatic.net/common?type=o&size=...,전체 관람가,애니메이션,2023.06.14.,미국,109분,"불, 물, 공기, 흙 4개의 원소들이 살고 있는 ‘엘리멘트 시티’ 재치 있고 불처럼...",None,NaN
3,4,더 퍼스트 슬램덩크,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.01.04.,일본,124분,"전국 제패를 꿈꾸는 북산고 농구부 5인방의 꿈과 열정, 멈추지 않는 도전을 그린 영화",None,NaN
4,5,가디언즈 오브 갤럭시: Volume 3,https://search.pstatic.net/common?type=o&size=...,12세 관람가,"액션, 모험, SF",2023.05.03.,미국,150분,‘가모라’를 잃고 슬픔에 빠져 있던 ‘피터 퀼’이 위기에 처한 은하계와 동료를 지키...,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
622,623,잇 컴스 앳 나잇,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"미스터리, 스릴러",2017.10.19.,미국,92분,의문의 존재를 피해 외딴 숲에 집을 짓고 자신만의 규칙을 만들어 아내와 아들을 돌보...,None,NaN
623,624,퍼스널 쇼퍼,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"드라마, 스릴러",2017.02.09.,프랑스,105분,프랑스 파리에서 퍼스널 쇼퍼로 일하는 미국 여자 ‘모린’(크리스틴 스튜어트) 영혼들...,None,NaN
624,625,케빈에 대하여,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"드라마, 스릴러, 서스펜스",2012.07.26.,"영국, 미국",112분,자유로운 삶을 즐기던 여행가 에바에게 아들 케빈이 생기면서 그녀의 삶은 180도 달...,None,NaN
625,626,악당들,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"범죄, 액션",2023.05.31.,"이탈리아, 브라질",89분,익명의 고객이 의뢰한 일을 완수만 한다면 특별사면을 보장하겠다는 무법 변호사의 제안...,None,NaN


In [22]:
len(movie_df)

627

In [23]:
row_num = len(movie_df)

In [24]:
for index in tqdm(range(row_num)):
    try:
        # df_movie.loc[index, "synopsys"]: synopsys 컬럼의 index번째 줄 데이터를 synopsys에 대입
        synopsys = movie_df.loc[index, "synopsys"]
        
        # df.loc[index, "synopsys_clear"]: Synopsys_clear 컬럼의 index번째 줄에 대입
        movie_df.loc[index, "synopsys_clear"] = preprocessingText(synopsys)
    except Exception as e:
        # 에러 발생 시 결측치 대입
        movie_df.loc[index, "synopsys_clear"] = np.NaN

  0%|          | 0/627 [00:00<?, ?it/s]

C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\2672635914.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마 석도 마동석 새롭다 팀원 살인 사건 조사 사건 조사 중 마 석도 신종 마약 사건 연루 알 수사 확대 한편 마약 사건 배후 주성철 이준혁 은 계속 판 약 유통 일본 조직 리키 아오키 무 타카 한국 사건 규모 점점 더 나쁘다 놈 데 이유 없다 제한 없다 판도 시원하다 싹' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  movie_df.loc[index, "synopsys_clear"] = preprocessingText(synopsys)


In [25]:
movie_df

,num,title,poster,degree,genre,open_date,country,movie_time,synopsys,synopsys_vector,synopsys_clear
0,1,범죄도시3,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"범죄, 액션",2023.05.31.,대한민국,105분,"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 ...",None,대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마...
1,2,스즈메의 문단속,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.03.08.,일본,122분,“이 근처에 폐허 없니? 문을 찾고 있어” 규슈의 한적한 마을에 살고 있는 소녀 ‘...,None,이 근처 폐허 없다 문 있다 규슈 한적하다 마을 살 있다 소녀 스즈메 문 여행 중인...
2,3,엘리멘탈,https://search.pstatic.net/common?type=o&size=...,전체 관람가,애니메이션,2023.06.14.,미국,109분,"불, 물, 공기, 흙 4개의 원소들이 살고 있는 ‘엘리멘트 시티’ 재치 있고 불처럼...",None,불 물 공기 흙 개 원소 살 있다 엘리 멘트 시티 재치 있다 불 열정 넘치다 앰버 ...
3,4,더 퍼스트 슬램덩크,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.01.04.,일본,124분,"전국 제패를 꿈꾸는 북산고 농구부 5인방의 꿈과 열정, 멈추지 않는 도전을 그린 영화",None,전국 제패 북산 농구부 인방 꿈 열정 도전 그린 영화
4,5,가디언즈 오브 갤럭시: Volume 3,https://search.pstatic.net/common?type=o&size=...,12세 관람가,"액션, 모험, SF",2023.05.03.,미국,150분,‘가모라’를 잃고 슬픔에 빠져 있던 ‘피터 퀼’이 위기에 처한 은하계와 동료를 지키...,None,가모라 를 슬픔 있다 피터 퀼 이 위기 처 은하계 동료 위해 다시 한번 가디언즈 팀...
...,...,...,...,...,...,...,...,...,...,...,...
622,623,잇 컴스 앳 나잇,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"미스터리, 스릴러",2017.10.19.,미국,92분,의문의 존재를 피해 외딴 숲에 집을 짓고 자신만의 규칙을 만들어 아내와 아들을 돌보...,None,의문 존재 피해 외딴 숲 집 자신 규칙 아내 아들 남자 날 가족 그 집 은신 정체 ...
623,624,퍼스널 쇼퍼,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"드라마, 스릴러",2017.02.09.,프랑스,105분,프랑스 파리에서 퍼스널 쇼퍼로 일하는 미국 여자 ‘모린’(크리스틴 스튜어트) 영혼들...,None,프랑스 파리 퍼스 널 쇼퍼 일 미국 여자 린 크리스틴 스튜어트 영혼 대화 수 있다 ...
624,625,케빈에 대하여,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"드라마, 스릴러, 서스펜스",2012.07.26.,"영국, 미국",112분,자유로운 삶을 즐기던 여행가 에바에게 아들 케빈이 생기면서 그녀의 삶은 180도 달...,None,자유롭다 삶 여행가 에바 아들 케빈 그녀 도 일과 양육 동시 에바 케빈 이유 반항 ...
625,626,악당들,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"범죄, 액션",2023.05.31.,"이탈리아, 브라질",89분,익명의 고객이 의뢰한 일을 완수만 한다면 특별사면을 보장하겠다는 무법 변호사의 제안...,None,익명 고객 의뢰 일 완수 특별사면 보장 무법 변호사 제안 알제리 감옥 가석방 빅 산...


In [26]:
# df['synopsys_clear']의 마지막에 " " 추가
movie_df["synopsys_clear"] = movie_df["synopsys_clear"].astype(str) + " "

In [27]:
# 공백을 기준으로 단어별로 분리해서 리턴
movie_df["synopsys_clear_list"] = movie_df["synopsys_clear"].apply(lambda data: data.split(" "))

In [28]:
movie_df

,num,title,poster,degree,genre,open_date,country,movie_time,synopsys,synopsys_vector,synopsys_clear,synopsys_clear_list
0,1,범죄도시3,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"범죄, 액션",2023.05.31.,대한민국,105분,"대체불가 괴물형사 마석도, 서울 광수대로 발탁! 베트남 납치 살해범 검거 후 7년 ...",None,대체 불가 괴물 형사 마 석도 서울 광수 발탁 베트남 납치 살해 범 검거 후 뒤 마...,"[대체, 불가, 괴물, 형사, 마, 석도, 서울, 광수, 발탁, 베트남, 납치, 살..."
1,2,스즈메의 문단속,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.03.08.,일본,122분,“이 근처에 폐허 없니? 문을 찾고 있어” 규슈의 한적한 마을에 살고 있는 소녀 ‘...,None,이 근처 폐허 없다 문 있다 규슈 한적하다 마을 살 있다 소녀 스즈메 문 여행 중인...,"[이, 근처, 폐허, 없다, 문, 있다, 규슈, 한적하다, 마을, 살, 있다, 소녀..."
2,3,엘리멘탈,https://search.pstatic.net/common?type=o&size=...,전체 관람가,애니메이션,2023.06.14.,미국,109분,"불, 물, 공기, 흙 4개의 원소들이 살고 있는 ‘엘리멘트 시티’ 재치 있고 불처럼...",None,불 물 공기 흙 개 원소 살 있다 엘리 멘트 시티 재치 있다 불 열정 넘치다 앰버 ...,"[불, 물, 공기, 흙, 개, 원소, 살, 있다, 엘리, 멘트, 시티, 재치, 있다..."
3,4,더 퍼스트 슬램덩크,https://search.pstatic.net/common?type=o&size=...,12세 관람가,애니메이션,2023.01.04.,일본,124분,"전국 제패를 꿈꾸는 북산고 농구부 5인방의 꿈과 열정, 멈추지 않는 도전을 그린 영화",None,전국 제패 북산 농구부 인방 꿈 열정 도전 그린 영화,"[전국, 제패, 북산, 농구부, 인방, 꿈, 열정, 도전, 그린, 영화, ]"
4,5,가디언즈 오브 갤럭시: Volume 3,https://search.pstatic.net/common?type=o&size=...,12세 관람가,"액션, 모험, SF",2023.05.03.,미국,150분,‘가모라’를 잃고 슬픔에 빠져 있던 ‘피터 퀼’이 위기에 처한 은하계와 동료를 지키...,None,가모라 를 슬픔 있다 피터 퀼 이 위기 처 은하계 동료 위해 다시 한번 가디언즈 팀...,"[가모라, 를, 슬픔, 있다, 피터, 퀼, 이, 위기, 처, 은하계, 동료, 위해,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
622,623,잇 컴스 앳 나잇,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"미스터리, 스릴러",2017.10.19.,미국,92분,의문의 존재를 피해 외딴 숲에 집을 짓고 자신만의 규칙을 만들어 아내와 아들을 돌보...,None,의문 존재 피해 외딴 숲 집 자신 규칙 아내 아들 남자 날 가족 그 집 은신 정체 ...,"[의문, 존재, 피해, 외딴, 숲, 집, 자신, 규칙, 아내, 아들, 남자, 날, ..."
623,624,퍼스널 쇼퍼,https://search.pstatic.net/common?type=o&size=...,15세 관람가,"드라마, 스릴러",2017.02.09.,프랑스,105분,프랑스 파리에서 퍼스널 쇼퍼로 일하는 미국 여자 ‘모린’(크리스틴 스튜어트) 영혼들...,None,프랑스 파리 퍼스 널 쇼퍼 일 미국 여자 린 크리스틴 스튜어트 영혼 대화 수 있다 ...,"[프랑스, 파리, 퍼스, 널, 쇼퍼, 일, 미국, 여자, 린, 크리스틴, 스튜어트,..."
624,625,케빈에 대하여,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"드라마, 스릴러, 서스펜스",2012.07.26.,"영국, 미국",112분,자유로운 삶을 즐기던 여행가 에바에게 아들 케빈이 생기면서 그녀의 삶은 180도 달...,None,자유롭다 삶 여행가 에바 아들 케빈 그녀 도 일과 양육 동시 에바 케빈 이유 반항 ...,"[자유롭다, 삶, 여행가, 에바, 아들, 케빈, 그녀, 도, 일과, 양육, 동시, ..."
625,626,악당들,https://search.pstatic.net/common?type=o&size=...,청소년 관람불가,"범죄, 액션",2023.05.31.,"이탈리아, 브라질",89분,익명의 고객이 의뢰한 일을 완수만 한다면 특별사면을 보장하겠다는 무법 변호사의 제안...,None,익명 고객 의뢰 일 완수 특별사면 보장 무법 변호사 제안 알제리 감옥 가석방 빅 산...,"[익명, 고객, 의뢰, 일, 완수, 특별사면, 보장, 무법, 변호사, 제안, 알제리..."


In [29]:
word2vec = Word2Vec(movie_df["synopsys_clear_list"], # iterable
                    sg=1, # 0: CBOW, 1: Skip-gram
                    vector_size=80, # WordVector의 열의 수
                    window=3, # 고려할 앞뒤 폭 (이 경우는 앞뒤 세 개의 단어)
                    min_count=2, # 사용할 단어의 최소 빈도(2회 이하의 단어는 무시)
                    workers=4 # 동시에 처리할 작업 수 (CPU 코어 개수와 비슷하게 설정하세요.)
                    )

In [30]:
# Vector 형태로 변환한 단어 리스트 조회
word2vec.wv.key_to_index

{'': 0,
 '그': 1,
 '있다': 2,
 '시작': 3,
 '수': 4,
 '위해': 5,
 '없다': 6,
 '것': 7,
 '사람': 8,
 '자신': 9,
 '이': 10,
 '의': 11,
 '사랑': 12,
 '날': 13,
 '은': 14,
 '그녀': 15,
 '두': 16,
 '를': 17,
 '속': 18,
 '세상': 19,
 '친구': 20,
 '알': 21,
 '가족': 22,
 '과': 23,
 '새롭다': 24,
 '같다': 25,
 '모든': 26,
 '곳': 27,
 '꿈': 28,
 '사건': 29,
 '세계': 30,
 '전': 31,
 '엄마': 32,
 '그렇다': 33,
 '살': 34,
 '마음': 35,
 '집': 36,
 '이야기': 37,
 '위': 38,
 '다시': 39,
 '서로': 40,
 '과연': 41,
 '중': 42,
 '아이': 43,
 '삶': 44,
 '마을': 45,
 '일': 46,
 '후': 47,
 '시간': 48,
 '남자': 49,
 '영화': 50,
 '뒤': 51,
 '앞': 52,
 '온': 53,
 '다른': 54,
 '향': 55,
 '못': 56,
 '딸': 57,
 '위기': 58,
 '순간': 59,
 '하나': 60,
 '사이': 61,
 '아들': 62,
 '구': 63,
 '소녀': 64,
 '가장': 65,
 '모두': 66,
 '학교': 67,
 '더': 68,
 '내': 69,
 '인생': 70,
 '아내': 71,
 '비밀': 72,
 '운명': 73,
 '우리': 74,
 '한편': 75,
 '기억': 76,
 '마지막': 77,
 '말': 78,
 '단': 79,
 '채': 80,
 '힘': 81,
 '지구': 82,
 '안': 83,
 '사실': 84,
 '하루': 85,
 '발견': 86,
 '이자': 87,
 '세': 88,
 '둘': 89,
 '만난': 90,
 '대한': 91,
 '상황': 92,
 '점점': 93,
 '남편': 94,
 

In [31]:
# Vector 형태로 변환한 단어만 조회
word2vec.wv.key_to_index.keys()

dict_keys(['', '그', '있다', '시작', '수', '위해', '없다', '것', '사람', '자신', '이', '의', '사랑', '날', '은', '그녀', '두', '를', '속', '세상', '친구', '알', '가족', '과', '새롭다', '같다', '모든', '곳', '꿈', '사건', '세계', '전', '엄마', '그렇다', '살', '마음', '집', '이야기', '위', '다시', '서로', '과연', '중', '아이', '삶', '마을', '일', '후', '시간', '남자', '영화', '뒤', '앞', '온', '다른', '향', '못', '딸', '위기', '순간', '하나', '사이', '아들', '구', '소녀', '가장', '모두', '학교', '더', '내', '인생', '아내', '비밀', '운명', '우리', '한편', '기억', '마지막', '말', '단', '채', '힘', '지구', '안', '사실', '하루', '발견', '이자', '세', '둘', '만난', '대한', '상황', '점점', '남편', '과거', '정체', '최고', '팀', '끝', '특별하다', '아버지', '유일하다', '여자', '처', '로', '소년', '이름', '간다', '마주', '작전', '생활', '때', '죽음', '인간', '건', '경찰', '조직', '처음', '나', '준비', '도시', '누구', '현실', '감정', '살인', '이제', '결심', '이후', '모험', '존재', '지금', '인하다', '무엇', '사고', '생각', '도움', '문', '아빠', '홀로', '무대', '뜨겁다', '관계', '일상', '수사', '결혼', '눈', '길', '모습', '카', '전쟁', '도착', '밤', '등', '어떻다', '여행', '분', '선택', '희망', '새', '당하다', '진실', '리', '연인', '작다', '목숨', '형사', '아름답다', '바로', '동생', '잠시', '금',

In [32]:
# 빈도수 순위 0이 가장 빈도수가 높은 단어입니다.
# 액션의 인덱스
word2vec.wv.key_to_index['액션']

314

In [33]:
# 로맨스의 인덱스
word2vec.wv.key_to_index['로맨스']

546

In [34]:
# 액션과 비슷한 벡터를 가진 단어 조회
word2vec.wv.most_similar('액션')

[('마지막', 0.9985154271125793),
 ('약속', 0.9984884858131409),
 ('정체', 0.998435914516449),
 ('뜨겁다', 0.9984268546104431),
 ('안', 0.9983968734741211),
 ('명의', 0.9983614683151245),
 ('때', 0.9983446598052979),
 ('형사', 0.9983441233634949),
 ('후', 0.9983271360397339),
 ('학교', 0.9983208775520325)]

In [35]:
# 로맨스와 비슷한 벡터를 가진 단어 조회
word2vec.wv.most_similar('로맨스')

[('범죄', 0.9974051713943481),
 ('순간', 0.9972437620162964),
 ('연인', 0.9970911741256714),
 ('살기', 0.9969679117202759),
 ('형제', 0.9968899488449097),
 ('길', 0.9968771934509277),
 ('위해', 0.9968250393867493),
 ('정체', 0.9968132972717285),
 ('철수', 0.99680095911026),
 ('모습', 0.9967950582504272)]

In [36]:
# 액션의 WordVector 조회
word2vec.wv['액션']

array([-8.3964370e-02, -3.2936356e-03,  1.9856974e-01,  1.7691973e-01,
       -3.0500876e-02, -4.8518978e-02,  2.4741383e-02, -1.2787312e-01,
       -5.6916632e-02,  9.6192300e-02, -1.1167439e-01, -1.3911782e-01,
       -7.5171880e-02,  1.0794555e-01,  1.5149850e-01,  1.9858262e-01,
       -2.7884477e-01, -5.6867614e-02, -1.2515221e-02, -2.5727993e-01,
        6.3587618e-03,  1.4304245e-01,  1.7866115e-01, -4.0333081e-02,
        8.7191269e-02,  7.5216442e-02,  5.0973915e-02,  8.8940278e-02,
       -2.5533032e-01, -1.0934148e-01,  2.1180217e-01, -6.3318565e-02,
        1.3595842e-01, -7.4672364e-02,  9.5549777e-02,  2.3914799e-01,
        3.1485856e-01, -7.7623144e-02, -1.8969651e-01, -7.8011163e-02,
       -1.7896879e-01,  1.0936630e-01, -2.3289187e-01,  3.0702088e-02,
        2.1443291e-01,  1.6009189e-03, -8.9111187e-02,  1.5594544e-01,
        1.3992786e-01,  7.5103894e-02,  2.3505040e-01,  6.4378478e-02,
        2.2488435e-01, -2.5539588e-02,  7.4499138e-02,  8.6829932e-03,
      

In [37]:
# WordVector로 변환한 단어들 조회
word2vec.wv.key_to_index.keys()

dict_keys(['', '그', '있다', '시작', '수', '위해', '없다', '것', '사람', '자신', '이', '의', '사랑', '날', '은', '그녀', '두', '를', '속', '세상', '친구', '알', '가족', '과', '새롭다', '같다', '모든', '곳', '꿈', '사건', '세계', '전', '엄마', '그렇다', '살', '마음', '집', '이야기', '위', '다시', '서로', '과연', '중', '아이', '삶', '마을', '일', '후', '시간', '남자', '영화', '뒤', '앞', '온', '다른', '향', '못', '딸', '위기', '순간', '하나', '사이', '아들', '구', '소녀', '가장', '모두', '학교', '더', '내', '인생', '아내', '비밀', '운명', '우리', '한편', '기억', '마지막', '말', '단', '채', '힘', '지구', '안', '사실', '하루', '발견', '이자', '세', '둘', '만난', '대한', '상황', '점점', '남편', '과거', '정체', '최고', '팀', '끝', '특별하다', '아버지', '유일하다', '여자', '처', '로', '소년', '이름', '간다', '마주', '작전', '생활', '때', '죽음', '인간', '건', '경찰', '조직', '처음', '나', '준비', '도시', '누구', '현실', '감정', '살인', '이제', '결심', '이후', '모험', '존재', '지금', '인하다', '무엇', '사고', '생각', '도움', '문', '아빠', '홀로', '무대', '뜨겁다', '관계', '일상', '수사', '결혼', '눈', '길', '모습', '카', '전쟁', '도착', '밤', '등', '어떻다', '여행', '분', '선택', '희망', '새', '당하다', '진실', '리', '연인', '작다', '목숨', '형사', '아름답다', '바로', '동생', '잠시', '금',

In [38]:
# WordVector로 변환한 단어들 word2vec_words에 대입
word2vec_words = word2vec.wv.key_to_index.keys()

In [40]:
movie_df[["num", "synopsys_clear_list"]][0:3]

,num,synopsys_clear_list
0,1,"[대체, 불가, 괴물, 형사, 마, 석도, 서울, 광수, 발탁, 베트남, 납치, 살..."
1,2,"[이, 근처, 폐허, 없다, 문, 있다, 규슈, 한적하다, 마을, 살, 있다, 소녀..."
2,3,"[불, 물, 공기, 흙, 개, 원소, 살, 있다, 엘리, 멘트, 시티, 재치, 있다..."


In [42]:
for index in range(3):
    # 영화 번호, 제목, 줄거리 조회
    num = movie_df.loc[index, "num"]
    print("num =", num)
    title = movie_df.loc[index, "title"]
    print("title =", title)
    line = movie_df.loc[index, "synopsys_clear_list"]
    print("line =", line)
    
    doc2vec = None
    count = 0
    
    # 줄거리에서 단어 한 개씩 리턴
    for word in line:
        print("word =", end="\t")
        if word in word2vec_words:
            count += 1
            if doc2vec is None: # 첫번째 단어일 경우
                doc2vec = word2vec.wv[word] # 단어의 WordVector 값 대입
            else: # 첫번째 단어가 아닐 때
                doc2vec = doc2vec + word2vec.wv[word] # 단어의 WordVector 값 추가
    
    if doc2vec is not None:
        # 단어 벡터를 모두 더한 후 벡터의 값을 단어의 개수로 나눠줌
        doc2vec = doc2vec / count
    print("\n")
    print("=" * 30)
    print("num =", num, "title =", title, ": doc2vec =", doc2vec)
    print("=" * 30)    

num = 1
title = 범죄도시3
line = ['대체', '불가', '괴물', '형사', '마', '석도', '서울', '광수', '발탁', '베트남', '납치', '살해', '범', '검거', '후', '뒤', '마', '석도', '마동석', '새롭다', '팀원', '살인', '사건', '조사', '사건', '조사', '중', '마', '석도', '신종', '마약', '사건', '연루', '알', '수사', '확대', '한편', '마약', '사건', '배후', '주성철', '이준혁', '은', '계속', '판', '약', '유통', '일본', '조직', '리키', '아오키', '무', '타카', '한국', '사건', '규모', '점점', '더', '나쁘다', '놈', '데', '이유', '없다', '제한', '없다', '판도', '시원하다', '싹', '']
word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	word =	

num = 1 title = 범죄도시3 : doc2vec = [-0.09031742 -0.00438213  0.18374924  0.165388

## 영화 줄거리 벡터 데이터베이스 저장

In [45]:
# 범죄도시 줄거리 조회
line = movie_df.loc[0, "synopsys_clear_list"]
print("line =", line)
print("=" * 30)
num = movie_df.loc[0, "num"]
title = movie_df.loc[0, "title"]
line = movie_df.loc[0, "synopsys_clear_list"]

doc2vec = None
count = 0
for word in line:
    print("word =", word, end="\t")
    if word in word2vec_words:
        count += 1
        if doc2vec is None: # 첫번째 단어일 경우
            doc2vec = word2vec.wv[word] # 단어의 WordVector 값 대입
        else: # 첫번째 단어가 아닐 때
            doc2vec = doc2vec + word2vec.wv[word] # 단어의 WordVector 값 추가
    
if doc2vec is not None:
    # 단어 벡터를 모두 더한 후 벡터의 값을 단어의 개수로 나눠줌
    doc2vec = doc2vec / count
print("\n")
print("=" * 30)
print("num =", num, "title =", title, ": doc2vec =", doc2vec)

line = ['대체', '불가', '괴물', '형사', '마', '석도', '서울', '광수', '발탁', '베트남', '납치', '살해', '범', '검거', '후', '뒤', '마', '석도', '마동석', '새롭다', '팀원', '살인', '사건', '조사', '사건', '조사', '중', '마', '석도', '신종', '마약', '사건', '연루', '알', '수사', '확대', '한편', '마약', '사건', '배후', '주성철', '이준혁', '은', '계속', '판', '약', '유통', '일본', '조직', '리키', '아오키', '무', '타카', '한국', '사건', '규모', '점점', '더', '나쁘다', '놈', '데', '이유', '없다', '제한', '없다', '판도', '시원하다', '싹', '']
word = 대체	word = 불가	word = 괴물	word = 형사	word = 마	word = 석도	word = 서울	word = 광수	word = 발탁	word = 베트남	word = 납치	word = 살해	word = 범	word = 검거	word = 후	word = 뒤	word = 마	word = 석도	word = 마동석	word = 새롭다	word = 팀원	word = 살인	word = 사건	word = 조사	word = 사건	word = 조사	word = 중	word = 마	word = 석도	word = 신종	word = 마약	word = 사건	word = 연루	word = 알	word = 수사	word = 확대	word = 한편	word = 마약	word = 사건	word = 배후	word = 주성철	word = 이준혁	word = 은	word = 계속	word = 판	word = 약	word = 유통	word = 일본	word = 조직	word = 리키	word = 아오키	word = 무	word = 타카	word = 한국	word = 사건	word = 규모	word = 점점	word = 더	word = 나쁘다	wor

In [46]:
# 범죄도시 WordVector 조회
doc2vec

array([-0.09031742, -0.00438213,  0.18374924,  0.16538826, -0.02412419,
       -0.0470959 ,  0.02974622, -0.12232964, -0.04527491,  0.08479325,
       -0.09704755, -0.12917127, -0.08256174,  0.09547924,  0.13576634,
        0.19583872, -0.26011267, -0.0433127 , -0.00987795, -0.24539736,
        0.0039552 ,  0.14814906,  0.1762124 , -0.04518434,  0.08480633,
        0.06202807,  0.04661286,  0.08244299, -0.24747318, -0.10703791,
        0.19916959, -0.07042498,  0.1400972 , -0.07758047,  0.09929813,
        0.22552803,  0.29434082, -0.08233175, -0.17828742, -0.06823637,
       -0.18006451,  0.10662587, -0.2107329 ,  0.04361695,  0.19340701,
        0.00778458, -0.08826867,  0.14095488,  0.11980713,  0.06188657,
        0.22689323,  0.06999132,  0.20805623, -0.03047178,  0.0776408 ,
        0.01411061,  0.01662656, -0.12679866, -0.04072646, -0.14758755,
        0.03545768, -0.03184932, -0.09150796,  0.00616095, -0.29011187,
       -0.2404031 , -0.22238731,  0.19222763,  0.06519293,  0.23

In [47]:
doc2vec.tostring()

C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\1786304619.py:1: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  doc2vec.tostring()


b'W\xf8\xb8\xbd\xf1\x97\x8f\xbb\xc3(<>\x8a[)>\x1a\xa0\xc5\xbc\xa2\xe7@\xbdV\xae\xf3<\xf6\x87\xfa\xbd/r9\xbd\x15\xa8\xad=\xde\xc0\xc6\xbdyE\x04\xbe!\x16\xa9\xbd\x9e\x8a\xc3=U\x06\x0b>\xf2\x89H>}-\x85\xbe\xa9h1\xbd\x1c\xd7!\xbcrI{\xbe\xa4\x9a\x81;c\xb4\x17>\x06q4>6\x139\xbd\xf1\xae\xad=%\x11~=!\xed>=\xdf\xd7\xa8=\x9ci}\xbe\xb16\xdb\xbd\x1d\xf3K>\xf9:\x90\xbd\xa4u\x0f>\x82\xe2\x9e\xbd\xd1\\\xcb=\xd2\xf0f>\xd7\xb3\x96>\x8d\x9d\xa8\xbd\xfa\x906\xbe\x83\xbf\x8b\xbd\xd5b8\xbe\xaa^\xda=^\xcaW\xbe\xaf\xa72=}\x0cF>\xcb\x15\xff;4\xc6\xb4\xbdzV\x10>q]\xf5=\xc6|}=\xb3Vh>\x9cW\x8f=\xb1\x0cU>\xf4\x9f\xf9\xbc$\x02\x9f=20g<m4\x88<\x82\xd7\x01\xbe\xca\xd0&\xbd1!\x17\xbe\x13<\x11=ot\x02\xbd\x87h\xbb\xbd\xc7\xe1\xc9;\x8b\x89\x94\xbe;,v\xbe\x80\xb9c\xbeR\xd7D>\xdf\x83\x85=\xc0Bt>\xde\xcbw=M\xb5G\xbe$\x85\xe6\xbd\xcd=X=L]N=&\xa4\xa1\xbc,\xe9\xb2=\x0c\xdd\x8a\xbe\xc4\x1f\x1a\xbe\xb2j\xe1\xbd'

In [48]:
# wordvector를 문자열로 변환해서 string_array에 저장
string_array = doc2vec.tostring()

C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\3405044950.py:2: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  string_array = doc2vec.tostring()


In [49]:
numpy_array = np.fromstring(string_array, dtype='float32')

C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\3980458634.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  numpy_array = np.fromstring(string_array, dtype='float32')


In [50]:
numpy_array

array([-0.09031742, -0.00438213,  0.18374924,  0.16538826, -0.02412419,
       -0.0470959 ,  0.02974622, -0.12232964, -0.04527491,  0.08479325,
       -0.09704755, -0.12917127, -0.08256174,  0.09547924,  0.13576634,
        0.19583872, -0.26011267, -0.0433127 , -0.00987795, -0.24539736,
        0.0039552 ,  0.14814906,  0.1762124 , -0.04518434,  0.08480633,
        0.06202807,  0.04661286,  0.08244299, -0.24747318, -0.10703791,
        0.19916959, -0.07042498,  0.1400972 , -0.07758047,  0.09929813,
        0.22552803,  0.29434082, -0.08233175, -0.17828742, -0.06823637,
       -0.18006451,  0.10662587, -0.2107329 ,  0.04361695,  0.19340701,
        0.00778458, -0.08826867,  0.14095488,  0.11980713,  0.06188657,
        0.22689323,  0.06999132,  0.20805623, -0.03047178,  0.0776408 ,
        0.01411061,  0.01662656, -0.12679866, -0.04072646, -0.14758755,
        0.03545768, -0.03184932, -0.09150796,  0.00616095, -0.29011187,
       -0.2404031 , -0.22238731,  0.19222763,  0.06519293,  0.23

In [51]:
row_num = len(movie_df)

In [52]:
# DB 쿼리를 실행할 객체 생성
cursor = db.cursor()

In [53]:
for index in range(row_num):
    # 영화 번호, 제목, 줄거리 조회
    num = movie_df.loc[index, "num"]
    print("num =", num)
    title = movie_df.loc[index, "title"]
    print("title =", title)
    line = movie_df.loc[index, "synopsys_clear_list"]
    
    doc2vec = None
    count = 0
    
    # 줄거리에서 단어 한 개씩 리턴
    for word in line:
        if word in word2vec_words:
            count += 1
            if doc2vec is None: # 첫번째 단어일 경우
                doc2vec = word2vec.wv[word] # 단어의 WordVector 값 대입
            else: # 첫번째 단어가 아닐 때
                doc2vec = doc2vec + word2vec.wv[word] # 단어의 WordVector 값 추가
    
    if doc2vec is not None:
        # 단어 벡터를 모두 더한 후 벡터의 값을 단어의 개수로 나눠줌
        doc2vec = doc2vec / count
        
    # movie_tbl 테이블에 영화 줄거리 WordVector를 추가할 쿼리 생성
    sql = "UPDATE movie_tbl SET synopsys_vector=%s WHERE num=%s "
    
    # sql 쿼리 실행
    cursor.execute(sql, (doc2vec.tostring(), num))
    # 커밋 실행
    db.commit()
    print("=" * 30)    

num = 1
title = 범죄도시3
num = 2
title = 스즈메의 문단속
num = 3
title = 엘리멘탈
num = 4
title = 더 퍼스트 슬램덩크
num = 5
title = 가디언즈 오브 갤럭시: Volume 3
num = 6
title = 아바타: 물의 길
num = 7
title = 미션 임파서블: 데드 레코닝 PART ONE
num = 8
title = 슈퍼 마리오 브라더스
num = 9
title = 존 윅 4
num = 10
title = 영웅
num = 11
title = 분노의 질주: 라이드 오어 다이


C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\3142418904.py:29: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  cursor.execute(sql, (doc2vec.tostring(), num))


num = 12
title = 교섭
num = 13
title = 앤트맨과 와스프: 퀀텀매니아
num = 14
title = 스파이더맨: 어크로스 더 유니버스
num = 15
title = 인디아나 존스: 운명의 다이얼
num = 16
title = 장화신은 고양이: 끝내주는 모험
num = 17
title = 트랜스포머: 비스트의 서막
num = 18
title = 대외비
num = 19
title = 극장판 짱구는 못말려: 동물소환 닌자 배꼽수비대
num = 20
title = 리바운드
num = 21
title = 귀공자
num = 22
title = 유령
num = 23
title = 인어공주
num = 24
title = 플래시
num = 25
title = 귀멸의 칼날: 상현집결, 그리고 도공 마을로
num = 26
title = 명탐정코난: 흑철의 어영
num = 27
title = 타이타닉
num = 28
title = 오늘 밤, 세계에서 이 사랑이 사라진다 해도
num = 29
title = 서치 2
num = 30
title = 카운트
num = 31
title = 상견니
num = 32
title = 밀수
num = 33
title = 바비
num = 34
title = 웅남이
num = 35
title = 여름날 우리
num = 36
title = 인시디어스: 빨간 문
num = 37
title = 던전 앤 드래곤: 도적들의 명예
num = 38
title = 옥수역귀신
num = 39
title = 아임 히어로 더 파이널
num = 40
title = 메간
num = 41
title = 바빌론
num = 42
title = 멍뭉이
num = 43
title = 킬링 로맨스
num = 44
title = 마루이 비디오
num = 45
title = 남은 인생 10년
num = 46
title = 두다다쿵: 후후섬의 비밀
num = 47
title = 문재인입니다
num = 48
title = 다음 소희
num = 49
title = 에브리

In [54]:
# 영화 제목과 WordVector를 조회하는 쿼리
sql = "SELECT title, synopsys_vector FROM movie_tbl WHERE num=1"
cursor.execute(sql)
# 조회 결과를 rows에 대입
rows = cursor.fetchall()

In [55]:
for row in rows:
    print("title: row[0]  =", row[0])
    print("=" * 30)
    print("synopsys_vector: row[1] =", row[1])
    print("=" * 30)
    print("synopsys_vector: row[1] =", np.fromstring(row[1], dtype='float32'))

title: row[0]  = 범죄도시3
synopsys_vector: row[1] = b'W\xf8\xb8\xbd\xf1\x97\x8f\xbb\xc3(<>\x8a[)>\x1a\xa0\xc5\xbc\xa2\xe7@\xbdV\xae\xf3<\xf6\x87\xfa\xbd/r9\xbd\x15\xa8\xad=\xde\xc0\xc6\xbdyE\x04\xbe!\x16\xa9\xbd\x9e\x8a\xc3=U\x06\x0b>\xf2\x89H>}-\x85\xbe\xa9h1\xbd\x1c\xd7!\xbcrI{\xbe\xa4\x9a\x81;c\xb4\x17>\x06q4>6\x139\xbd\xf1\xae\xad=%\x11~=!\xed>=\xdf\xd7\xa8=\x9ci}\xbe\xb16\xdb\xbd\x1d\xf3K>\xf9:\x90\xbd\xa4u\x0f>\x82\xe2\x9e\xbd\xd1\\\xcb=\xd2\xf0f>\xd7\xb3\x96>\x8d\x9d\xa8\xbd\xfa\x906\xbe\x83\xbf\x8b\xbd\xd5b8\xbe\xaa^\xda=^\xcaW\xbe\xaf\xa72=}\x0cF>\xcb\x15\xff;4\xc6\xb4\xbdzV\x10>q]\xf5=\xc6|}=\xb3Vh>\x9cW\x8f=\xb1\x0cU>\xf4\x9f\xf9\xbc$\x02\x9f=20g<m4\x88<\x82\xd7\x01\xbe\xca\xd0&\xbd1!\x17\xbe\x13<\x11=ot\x02\xbd\x87h\xbb\xbd\xc7\xe1\xc9;\x8b\x89\x94\xbe;,v\xbe\x80\xb9c\xbeR\xd7D>\xdf\x83\x85=\xc0Bt>\xde\xcbw=M\xb5G\xbe$\x85\xe6\xbd\xcd=X=L]N=&\xa4\xa1\xbc,\xe9\xb2=\x0c\xdd\x8a\xbe\xc4\x1f\x1a\xbe\xb2j\xe1\xbd'
synopsys_vector: row[1] = [-0.09031742 -0.00438213  0.18374924  0.16

C:\Users\tjk\AppData\Local\Temp\ipykernel_17976\3871908814.py:6: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  print("synopsys_vector: row[1] =", np.fromstring(row[1], dtype='float32'))


In [56]:
# DB와 연결 종료
db.close()